In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
import matplotlib.pyplot as plt

# Load the Dataset
data_dir = "C:/Users/Administrator/Downloads/sign img/Image"  # Path to your ASL dataset folder
image_size = (300, 300)  # Increase image size to match EfficientNet's requirements

data_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Use EfficientNet's preprocessing function
    validation_split=0.2
)

train_data = data_gen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_data = data_gen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

num_classes = len(train_data.class_indices)
class_labels = list(train_data.class_indices.keys())

print(f"Number of classes: {num_classes}")
print(f"Class labels: {class_labels}")

# Define the Transfer Learning Model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
base_model.trainable = False  # Freeze the base model layers initially

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),  # Regularization to reduce overfitting
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')  # Output layer with softmax for multi-class classification
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model
epochs = 30
history = model.fit(train_data, validation_data=val_data, epochs=epochs)

# Save the Model
model.save("asl_model_efficientnet.keras")

# Load the Model for Real-time Detection
model = tf.keras.models.load_model("asl_model_efficientnet.keras")

# Set up OpenCV for Real-time Detection
cap = cv2.VideoCapture(0)  # Open the default camera

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()  # Capture frame
    if not ret:
        print("Failed to capture image")
        break

    # Preprocess the frame
    frame_resized = cv2.resize(frame, (300, 300))
    frame_preprocessed = preprocess_input(frame_resized)
    frame_reshaped = np.reshape(frame_preprocessed, (1, 300, 300, 3))

    # Make prediction
    predictions = model.predict(frame_reshaped)
    confidence = np.max(predictions[0])
    predicted_index = np.argmax(predictions[0])
    predicted_label = class_labels[predicted_index]

    # Display the predictions on the frame
    cv2.putText(frame, f'Predicted Class: {predicted_label} (Confidence: {confidence:.2f})', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('ASL Detection', frame)  # Show the frame with predictions

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and destroy all OpenCV windows
cap.release()
cv2.destroyAllWindows()


Found 575 images belonging to 6 classes.
Found 142 images belonging to 6 classes.
Number of classes: 6
Class labels: ['1', '2', '3', 'A', 'B', 'C']
Epoch 1/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 110s 4s/step - accuracy: 0.6881 - loss: 0.7953 - val_accuracy: 1.0000 - val_loss: 0.0347
Epoch 2/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - accuracy: 0.9919 - loss: 0.0417 - val_accuracy: 1.0000 - val_loss: 0.0140
Epoch 3/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - accuracy: 0.9941 - loss: 0.0184 - val_accuracy: 1.0000 - val_loss: 0.0046
Epoch 4/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 59s 3s/step - accuracy: 0.9934 - loss: 0.0162 - val_accuracy: 1.0000 - val_loss: 0.0020
Epoch 5/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 1.0000 - loss: 0.0063 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 6/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 57s 3s/step - accuracy: 0.9993 - loss: 0.0065 - val_accuracy: 1.0000 - val_loss: 0.0014
Epoch 7/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - accuracy: 1.0000 - loss: 0.0036 - val_accur